In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

In [238]:
desired_data = {'January' :pd.Series(['riya', 24, 'F'], index=['Name', 'Age','Status']),
                'February':  pd.Series(['john', 28, 'M'], index=['Name', 'Age','Status']),
                'March' : pd.Series(['alex', 34, 'M'], index=['Name', 'Age','Status'])}

df2 = pd.DataFrame(desired_data)
df2

,January,February,March
Name,riya,john,alex
Age,24,28,34
Status,F,M,M


In [254]:
df1.columns = df1.iloc[2]
df1 = df1.drop(df1.index[2])
df2 = df1.rename_axis(None).rename_axis(None,axis='columns')
df2

,February,January,March
Name,john,riya,alex
Age,28,24,34
Status,M,F,M


In [260]:
df = pd.DataFrame({
    'flag': {0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 0, 6: 0, 7: 1, 8: 1, 9: 1}
})

# Create Group IDs based on where 0s are
groups = df.flag.eq(0).cumsum()

# Groupby groups and transform each group to the size
#df['Count'] = df.groupby(groups)['flag'].transform('size')
#df


0    NaN
1    1.0
2    0.0
3    0.0
4    0.0
5   -1.0
6    0.0
7    1.0
8    0.0
9    0.0
dtype: float64

In [262]:
d="""IDBILL,IDBUYER,BILL,DATE
001,768787,45,1897-07-24
001,768787,67,1897-07-24
001,768787,98,1897-07-24
002,768787,30,1897-07-24
002,768787,15,1897-07-24
002,768787,12,1897-07-24
005,786545,45,1897-08-19
008,657676,89,1989-09-23
009,657676,42,1989-09-23
010,657676,18,1989-09-23
012,657676,51,1990-03-10
016,892354,73,1990-03-10
018,892354,48,1765-02-14
020,892354,62,1765-02-14"""
df=g(d)
df.DATE = pd.to_datetime(df.DATE)
df

,IDBILL,IDBUYER,BILL,DATE
0,1,768787,45,1897-07-24
1,1,768787,67,1897-07-24
2,1,768787,98,1897-07-24
3,2,768787,30,1897-07-24
4,2,768787,15,1897-07-24
5,2,768787,12,1897-07-24
6,5,786545,45,1897-08-19
7,8,657676,89,1989-09-23
8,9,657676,42,1989-09-23
9,10,657676,18,1989-09-23


In [263]:
df.groupby(['IDBUYER','DATE'],as_index=False, 
           sort=False)[['IDBILL','BILL']].min()

,IDBUYER,DATE,IDBILL,BILL
0,768787,1897-07-24,1,12
1,786545,1897-08-19,5,45
2,657676,1989-09-23,8,18
3,657676,1990-03-10,12,51
4,892354,1990-03-10,16,73
5,892354,1765-02-14,18,48


In [268]:
d="""Aerogenerador,Cause,DateOn,DateOFF
A0201,Windturbine Pause,02/01/2021 10:00,02/01/2021 10:05
A0202,Windturbine STOP,02/01/2021 10:00,02/01/2021 10:05
A0202,High Temperature DE,02/01/2021 10:00,02/01/2021 10:05"""
df=g(d)
df

,Aerogenerador,Cause,DateOn,DateOFF
0,A0201,Windturbine Pause,02/01/2021 10:00,02/01/2021 10:05
1,A0202,Windturbine STOP,02/01/2021 10:00,02/01/2021 10:05
2,A0202,High Temperature DE,02/01/2021 10:00,02/01/2021 10:05


In [269]:
df = df.query('Cause not in ["Windturbine Pause", "Windturbine STOP"]')
df

,Aerogenerador,Cause,DateOn,DateOFF
2,A0202,High Temperature DE,02/01/2021 10:00,02/01/2021 10:05


In [272]:
df[~df.Cause.str.contains("Windturbine")]

,Aerogenerador,Cause,DateOn,DateOFF
2,A0202,High Temperature DE,02/01/2021 10:00,02/01/2021 10:05


In [11]:
import re
name = 'AbcBBCTestVariable'
name = re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()
print(name)  # camel_case_name

abc_b_b_c_test_variable


In [16]:
data = {'Patient_admitted_id': [214321,224323,3234234,23423],
        'Date':  ['2021-01-22', '2021-01-22','2021-01-22','2021-01-20'], # This is just an example I have created here, the real X data frame contains proper date values generated with Datatime
        'Hospital_ID': ['1', '2','3','2'],
        }
X = pd.DataFrame(data, columns=['Patient_admitted_id','Date', 'Hospital_ID' ])
X

,Patient_admitted_id,Date,Hospital_ID
0,214321,2021-01-22,1
1,224323,2021-01-22,2
2,3234234,2021-01-22,3
3,23423,2021-01-20,2


In [19]:
X['Date']= pd.to_datetime(X['Date'])
df = X.groupby(pd.Grouper(key='Date', freq='D')).size().reset_index(name='Cases')       
df['Cumulative'] = df.Cases.cumsum()    
df

,Date,Cases,Cumulative
0,2021-01-20,1,1
1,2021-01-21,0,1
2,2021-01-22,3,4


In [3]:
d="""location,type,quarter
ny,aa,1
ny,aa,1
ny,bb,1
ny,aa,2
ny,aa,2
ny,bb,2
ca,cc,3
ca,aa,3"""
df=g(d)
df

,location,type,quarter
0,ny,aa,1
1,ny,aa,1
2,ny,bb,1
3,ny,aa,2
4,ny,aa,2
5,ny,bb,2
6,ca,cc,3
7,ca,aa,3


In [13]:
df.type + df.groupby(['location','type','quarter']).cumcount().add(1).astype(str).str.zfill(2)

0    aa01
1    aa02
2    bb01
3    aa01
4    aa02
5    bb01
6    cc01
7    aa01
dtype: object

In [14]:
d="""score,id_number, company_name,company_code,sal_date,action_reqd
20,IN2231D,AXN pvt Ltd,IN225,2019-12-22,Yes
45,UK654IN,Aviva Intl Ltd,IN115,2018-10-10,No
65,SL1432H,Ship Incorporations,CZ555,2015-08-19,Yes
35,LK0678G,Oppo Mobiles pvt ltd,PQ795,2018-06-26,Yes
59,NG5678J, Nokia Inc,RS885,2020-12-28,No
20,IN2231D,AXN pvt Ltd,IN215,2020-12-08,Yes"""
df1=g(d)
df1

,score,id_number,company_name,company_code,sal_date,action_reqd
0,20,IN2231D,AXN pvt Ltd,IN225,2019-12-22,Yes
1,45,UK654IN,Aviva Intl Ltd,IN115,2018-10-10,No
2,65,SL1432H,Ship Incorporations,CZ555,2015-08-19,Yes
3,35,LK0678G,Oppo Mobiles pvt ltd,PQ795,2018-06-26,Yes
4,59,NG5678J,Nokia Inc,RS885,2020-12-28,No
5,20,IN2231D,AXN pvt Ltd,IN215,2020-12-08,Yes


In [70]:
d="""Number,Sex
0,Female
1,Male
2,Not Available
3,Male
4,Not Available"""
df=g(d)
df

,Number,Sex
0,0,Female
1,1,Male
2,2,Not Available
3,3,Male
4,4,Not Available


In [320]:
d="""apple|orange|banana
(2021-01-01 00:00:00, 2)|(2021-01-02 00:00:00, 1)|(2021-01-01 00:00:00, 4)
(2021-01-02 00:00:00, 5)|(2021-01-04 00:00:00, 2)|(2021-01-03 00:00:00, 6)
(2021-01-03 00:00:00, 8)|(2021-01-06 00:00:00, 3)|(2021-01-04 00:00:00, 7)"""
df=pd.read_csv(StringIO(d), engine='python', sep='|')
df

,apple,orange,banana
0,"(2021-01-01 00:00:00, 2)","(2021-01-02 00:00:00, 1)","(2021-01-01 00:00:00, 4)"
1,"(2021-01-02 00:00:00, 5)","(2021-01-04 00:00:00, 2)","(2021-01-03 00:00:00, 6)"
2,"(2021-01-03 00:00:00, 8)","(2021-01-06 00:00:00, 3)","(2021-01-04 00:00:00, 7)"


In [346]:
df=DataFrame({'val':[['blue','green','white','yellow','orange','pink','black'],
['green','yellow','orange','pink','pink'],
['white','orange','black'],
['green','white','yellow','orange'],
['green']]})
df

,val
0,"[blue, green, white, yellow, orange, pink, black]"
1,"[green, yellow, orange, pink, pink]"
2,"[white, orange, black]"
3,"[green, white, yellow, orange]"
4,[green]


In [347]:
#df.apply(lambda x:  True if ['yellow'] in x else False, axis=1)
df['val'].apply(lambda x:[y for y in x if y=='yellow' or y=='green' or y=='black'])

0    [green, yellow, black]
1           [green, yellow]
2                   [black]
3           [green, yellow]
4                   [green]
Name: val, dtype: object

In [351]:
df.val.apply(lambda x: [color for color in x if col in ['yellow', 'green', 'black']])

0    [green, yellow, black]
1           [green, yellow]
2                   [black]
3           [green, yellow]
4                   [green]
Name: val, dtype: object

In [120]:
dates = pd.date_range("2020", "2021")
index = dates
index_list = list(index)
index_set = set(index)
type(index) ## pandas.core.indexes.datetimes.DatetimeIndex
type(index_list) ## list
type(index_set) ## set

set

In [121]:
%timeit [date in index for date in dates]

4.2 ms ± 50.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [122]:
%timeit [date in index_list for date in dates]

4.41 ms ± 32.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [123]:
%timeit [date in index_set for date in dates]

380 µs ± 8.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [93]:
%timeit index[120]

10.7 µs ± 332 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [96]:
%timeit pd.Timestamp('2020-04-30 00:00:00') in index_set

3.17 µs ± 31.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [124]:
d="""id,q
1,hello?
2,helloWorld
3,hi"""
df=g(d)
df

,id,q
0,1,hello?
1,2,helloWorld
2,3,hi


In [250]:
d="""MovieID|movieCater|rating
1|Action,Comedy,Adventure|4
2|Action,Crime|3
3|Crime|2"""
df=pd.read_csv(StringIO(d), engine='python', sep='|')
df

,MovieID,movieCater,rating
0,1,"Action,Comedy,Adventure",4
1,2,"Action,Crime",3
2,3,Crime,2


In [255]:
df.join(df['movieCater'].str.get_dummies(",").mul(df.rating,0))

,MovieID,movieCater,rating,Action,Adventure,Comedy,Crime
0,1,"Action,Comedy,Adventure",4,4,4,4,0
1,2,"Action,Crime",3,3,0,0,3
2,3,Crime,2,0,0,0,2


In [186]:
(df.drop("rating",1).join(df['movieCater'].str.get_dummies(",").mul(df['rating'],axis=0)))

,MovieID,movieCater,Action,Adventure,Comedy,Crime
0,1,"Action,Comedy,Adventure",4,4,4,0
1,2,"Action,Crime",3,0,0,3
2,3,Crime,0,0,0,2


In [234]:
s = [np.nan, 0, np.nan, 1, 0, np.nan,1, np.nan]
df = pd.DataFrame(s, columns=["s"])
df

,s
0,NaN
1,0.0
2,NaN
3,1.0
4,0.0
5,NaN
6,1.0
7,NaN


In [235]:
sf = df.ffill().values[:, 0]
desired = np.where(np.isnan(s) & (sf==0), sf, s)
desired

array([nan,  0.,  0.,  1.,  0.,  0.,  1., nan])

In [242]:
df.ffill().values[:,0]
np.isnan(s)& (sf==0)

array([False, False,  True, False, False,  True, False, False])

In [262]:
df1 = pd.DataFrame({
    'date':pd.date_range('2021-05-17', periods=6),
    'px':np.random.rand(6)
})
df1

,date,px
0,2021-05-17,0.244446
1,2021-05-18,0.183900
2,2021-05-19,0.288642
3,2021-05-20,0.031586
4,2021-05-21,0.575367
5,2021-05-22,0.380052


In [263]:
df2 = pd.DataFrame({
    'date':pd.date_range('2021-05-17', periods=5, freq='2D'),
    'px':np.random.rand(5)
})
df2

,date,px
0,2021-05-17,0.739383
1,2021-05-19,0.450789
2,2021-05-21,0.486184
3,2021-05-23,0.064725
4,2021-05-25,0.476046


In [264]:
df1[df1.date.isin(df2.date)]

,date,px
0,2021-05-17,0.244446
2,2021-05-19,0.288642
4,2021-05-21,0.575367


In [267]:
df2[0:3]

,date,px
0,2021-05-17,0.739383
1,2021-05-19,0.450789
2,2021-05-21,0.486184


In [268]:
import pandas as pd
date_1 = pd.date_range('2015-02-24', periods=5, freq='1T')
date_2 = pd.date_range('2015-02-25', periods=5, freq='1T')
date = date_1.union(date_2)
values = range(len(date))

df = pd.DataFrame({'date': date, 'values': values})
df

,date,values
0,2015-02-24 00:00:00,0
1,2015-02-24 00:01:00,1
2,2015-02-24 00:02:00,2
3,2015-02-24 00:03:00,3
4,2015-02-24 00:04:00,4
5,2015-02-25 00:00:00,5
6,2015-02-25 00:01:00,6
7,2015-02-25 00:02:00,7
8,2015-02-25 00:03:00,8
9,2015-02-25 00:04:00,9


In [287]:
d="""Date,Portfoliovalue
2021-05-01,50000.0
2021-05-05,52304.0"""
df=g(d)
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date').asfreq('D').reset_index()
df

,Date,Portfoliovalue
0,2021-05-01,50000.0
1,2021-05-02,NaN
2,2021-05-03,NaN
3,2021-05-04,NaN
4,2021-05-05,52304.0


In [290]:
df = pd.DataFrame({'id':[1,2,3,4,5],'firstname':['John','Gerry','Sarah','Pierre','Angela'],
'lastname': ['Doe','Ronald','Lee','Sam','Perez']})
df.to_dict(orient='records')

[{'id': 1, 'firstname': 'John', 'lastname': 'Doe'},
 {'id': 2, 'firstname': 'Gerry', 'lastname': 'Ronald'},
 {'id': 3, 'firstname': 'Sarah', 'lastname': 'Lee'},
 {'id': 4, 'firstname': 'Pierre', 'lastname': 'Sam'},
 {'id': 5, 'firstname': 'Angela', 'lastname': 'Perez'}]

In [298]:
df = pd.DataFrame({"A": [1, 1, 1, 2, 2],
                   "B": ["apple", "apple", "banana", "pineapple", "pineapple"],
                   "C": [[6, 5, 2], [2, 10, 2], [5, 37, 1], [4, 19, 2], [1, 5, 1]]})
df

,A,B,C
0,1,apple,"[6, 5, 2]"
1,1,apple,"[2, 10, 2]"
2,1,banana,"[5, 37, 1]"
3,2,pineapple,"[4, 19, 2]"
4,2,pineapple,"[1, 5, 1]"


In [301]:
xpos = pd.DataFrame({'N': [1, 2, 3, 4],'Type': ['External', 'External', 
                                                'Internal', 'Internal']})
xpos

,N,Type
0,1,External
1,2,External
2,3,Internal
3,4,Internal


In [312]:
L = pd.DataFrame({'N': [1, 2, 3, 4],'Length': [100, 300, 400, 200]})
L

,N,Length
0,1,100
1,2,300
2,3,400
3,4,200


In [315]:
L['Length'] = np.where(L.merge(xpos, on='N', how='inner').Type == 'External',np.nan,L.Length)

In [316]:
L

,N,Length
0,1,NaN
1,2,NaN
2,3,400.0
3,4,200.0


In [320]:
d="""ID,Date,City,PRCP,TAVG,TMAX,TMIN
abcd1,2020-01-01,Zurich,0.0,-1.90,-0.9,0.36
abcd1,2020-01-02,Zurich,9.1,1.96,12.7,4.90
abcd1,2020-01-04,Zurich,0.0,4.10,10.8,-2.60"""
df=g(d)
df

,ID,Date,City,PRCP,TAVG,TMAX,TMIN
0,abcd1,2020-01-01,Zurich,0.0,-1.90,-0.9,0.36
1,abcd1,2020-01-02,Zurich,9.1,1.96,12.7,4.90
2,abcd1,2020-01-04,Zurich,0.0,4.10,10.8,-2.60


In [321]:
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date')
df = df.asfreq('D').reset_index()

In [322]:
df

,Date,ID,City,PRCP,TAVG,TMAX,TMIN
0,2020-01-01,abcd1,Zurich,0.0,-1.90,-0.9,0.36
1,2020-01-02,abcd1,Zurich,9.1,1.96,12.7,4.90
2,2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,abcd1,Zurich,0.0,4.10,10.8,-2.60


In [344]:
d="""Id|Villas|Price_nospace
3|Peace|220,000
3|Peace|350,000
2|Rosa|270,000
1|Beach|250,000
0|Palm|230,000"""
df=pd.read_csv(StringIO(d), engine='python', sep='|')
df

,Id,Villas,Price_nospace
0,3,Peace,"220,000"
1,3,Peace,"350,000"
2,2,Rosa,"270,000"
3,1,Beach,"250,000"
4,0,Palm,"230,000"


In [345]:
df['Price_new'] = df['Price_nospace'].str.replace(',','',regex=True).astype(int)
df = df.sort_values(by='Price_new', ascending=False)
df['Price_nospace'] = df['Price_nospace'].str.replace(',',' ',regex=True)
df = df.drop(columns='Price_new').reset_index(drop=True)
df

,Id,Villas,Price_nospace
0,3,Peace,350 000
1,2,Rosa,270 000
2,1,Beach,250 000
3,0,Palm,230 000
4,3,Peace,220 000


In [341]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 1 to 0
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             5 non-null      int64 
 1   Villas         5 non-null      object
 2   Price_nospace  5 non-null      object
dtypes: int64(1), object(2)
memory usage: 160.0+ bytes


In [372]:
df = pd.DataFrame({"A": [1, 1, 1, 2, 2],
                       "B": ["apple", "apple", "banana", "pineapple", "pineapple"],
                       "C": [[6, 5, 2], [2, 10, 2], [5, 37, 1], [4, 19, 2], [1, 5, 1]]})
df

,A,B,C
0,1,apple,"[6, 5, 2]"
1,1,apple,"[2, 10, 2]"
2,1,banana,"[5, 37, 1]"
3,2,pineapple,"[4, 19, 2]"
4,2,pineapple,"[1, 5, 1]"


In [4]:
dict0 = {'date': ['1/1/2020', '1/1/2020', '1/1/2020', '1/1/2020', '1/1/2020', '1/2/2020', '1/2/2020', '1/2/2020', '1/2/2020',
                  '1/2/2020', '1/3/2020', '1/3/2020', '1/3/2020', '1/3/2020', '1/3/2020'],
         'ticker': ['SPY', 'AAPL', 'AMZN', 'XLK', 'XLY', 'SPY', 'AAPL', 'AMZN', 'XLK', 'XLY', 'SPY', 'AAPL', 'AMZN', 'XLK', 'XLY'],
         'returns': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5],
        'sector': [np.NaN, 'Tech', 'Cons Disc', np.NaN, np.NaN, np.NaN, 'Tech', 'Cons Disc', np.NaN, np.NaN, np.NaN, 'Tech', 'Cons Disc', np.NaN, np.NaN,]}
df = pd.DataFrame(dict0)
df.date = pd.to_datetime(df.date, format='%d/%m/%Y')
df = df.set_index(['date', 'ticker'])
df

returns     sector
date       ticker                    
2020-01-01 SPY           1        NaN
           AAPL          2       Tech
           AMZN          3  Cons Disc
           XLK           4        NaN
           XLY           5        NaN
2020-02-01 SPY           6        NaN
           AAPL          7       Tech
           AMZN          8  Cons Disc
           XLK           9        NaN
           XLY          10        NaN
2020-03-01 SPY           1        NaN
           AAPL          2       Tech
           AMZN          3  Cons Disc
           XLK           4        NaN
           XLY           5        NaN

In [25]:
dict0 = {'date': ['1/1/2020', '1/1/2020', '1/1/2020', '1/2/2020', '1/2/2020',
                  '1/2/2020', '1/3/2020', '1/3/2020', '1/3/2020'],
         'ticker': ['SPY', 'AAPL', 'MSFT', 'SPY', 'AAPL', 'MSFT', 'SPY', 'AAPL', 'MSFT'],
         'returns': [1, 2, 3, 4, 5, 6, 7, 8, 9]}
df = pd.DataFrame(dict0)

## Firstly I would convert date to datetime type
df.date = pd.to_datetime(df.date, format='%d/%m/%Y')
df['excess_returns'] = df.returns - df.groupby('date').returns.transform('first')

## by any means if I require to set the index to date and ticker
df = df.set_index(['date', 'ticker']) ## Only if required

df

returns  excess_returns
date       ticker                         
2020-01-01 SPY           1               0
           AAPL          2               1
           MSFT          3               2
2020-02-01 SPY           4               0
           AAPL          5               1
           MSFT          6               2
2020-03-01 SPY           7               0
           AAPL          8               1
           MSFT          9               2

In [21]:
def func(row):
    print(row)
    date, asset = row.name
    return df.loc[(date, asset), 'returns'] - df.loc[(date, 'SPY'), 'returns']



dict0 = {'date': ['1/1/2020', '1/1/2020', '1/1/2020', '1/2/2020', '1/2/2020',
                  '1/2/2020', '1/3/2020', '1/3/2020', '1/3/2020'],
         'ticker': ['SPY', 'AAPL', 'MSFT', 'SPY', 'AAPL', 'MSFT', 'SPY', 'AAPL', 'MSFT'],
         'returns': [1, 2, 3, 4, 5, 6, 7, 8, 9]}
df = pd.DataFrame(dict0)  ###
df = df.set_index(['date', 'ticker'])
df['excess_returns'] = df.apply(func, axis=1)

returns    1
Name: (1/1/2020, SPY), dtype: int64
returns    2
Name: (1/1/2020, AAPL), dtype: int64
returns    3
Name: (1/1/2020, MSFT), dtype: int64
returns    4
Name: (1/2/2020, SPY), dtype: int64
returns    5
Name: (1/2/2020, AAPL), dtype: int64
returns    6
Name: (1/2/2020, MSFT), dtype: int64
returns    7
Name: (1/3/2020, SPY), dtype: int64
returns    8
Name: (1/3/2020, AAPL), dtype: int64
returns    9
Name: (1/3/2020, MSFT), dtype: int64


In [29]:
df1 = pd.DataFrame({
    'date':pd.date_range(start = '2020.04.01', periods=4)
})

df2 = pd.DataFrame({
    'date':pd.date_range(start = '2020.05.01', periods=5)
})

df1

,date
0,2020-04-01
1,2020-04-02
2,2020-04-03
3,2020-04-04


In [41]:
import glob
dfdetails = pd.DataFrame(columns=['dataframe','start_date','end_date'])
for file in glob.glob("./*.csv"):
     df = pd.read_csv(file, header=None, names=['date'])
    df.date = pd.to_datetime(df.date)
    temp_df = pd.DataFrame({
        'dataframe': 'df'+str(count),
        'start_date': df.date.min(),
        'end_date': df.date.max()
    }, index=[0])
    dfdetails = dfdetails.append(temp_df)
    count += 1


  dataframe start_date   end_date
0       df1 2020-06-01 2020-06-03
  dataframe start_date   end_date
0       df2 2020-07-01 2020-07-02
  dataframe start_date   end_date
0       df3 2020-04-01 2020-04-04
  dataframe start_date   end_date
0       df4 2020-05-01 2020-05-05


In [42]:
dfdetails

,dataframe,start_date,end_date
0,df1,2020-06-01,2020-06-03
0,df2,2020-07-01,2020-07-02
0,df3,2020-04-01,2020-04-04
0,df4,2020-05-01,2020-05-05


In [46]:
pd.read_csv('test.csv', header=None, names=['date'])

,date
0,2020-04-01
1,2020-04-02
2,2020-04-03
3,2020-04-04


In [65]:
import glob
dfdetails = pd.DataFrame(columns=['dataframe','start_date','end_date'])
for file in glob.glob("./*.csv"): ## replace the path with 'F:\dfs'. Use * to read all files from a directory.
    df = pd.read_csv(file, header=None, names=['date']) ## when file does not contain header.
    df.date = pd.to_datetime(df.date)
    df_name = file.split('/')[-1]
    temp_df = pd.DataFrame({
        'dataframe': df_name,
        'start_date': df.date.min(),
        'end_date': df.date.max()
    }, index=[0])
    dfdetails = dfdetails.append(temp_df)

## Outside for loop
dfdetails = dfdetails.reset_index(drop=True)
dfdetails

,dataframe,start_date,end_date
0,foo.csv,2020-06-01,2020-06-03
1,temp.csv,2020-07-01,2020-07-02
2,test.csv,2020-04-01,2020-04-04
3,te.csv,2020-05-01,2020-05-05


In [63]:
dfdetails

,dataframe,start_date,end_date
0,foo.csv,2020-06-01,2020-06-03
0,temp.csv,2020-07-01,2020-07-02
0,test.csv,2020-04-01,2020-04-04
0,te.csv,2020-05-01,2020-05-05


In [58]:
for file in glob.glob("./*.csv"):
    fname = file.split('/')[-1]
    print(fname)

foo.csv
temp.csv
test.csv
te.csv


In [66]:
data = {'col_1': ['a', 'b', 'c', 'd','c'],'col_2': [3, 2, 1, 0, 4],'col3':[99,88,77,66,55]}
df = pd.DataFrame.from_dict(data)
df

,col_1,col_2,col3
0,a,3,99
1,b,2,88
2,c,1,77
3,d,0,66
4,c,4,55


In [69]:
df.groupby(['col_1'],as_index=False).agg(count=('col_2','count'))

,col_1,count
0,a,1
1,b,1
2,c,2
3,d,1


In [71]:
from pandas import Timestamp
d={'account_id': {0: 1, 1: 1, 2: 1, 3: 2, 4: 2, 5: 3},
 'contract_id': {0: 'AAA',
  1: 'ADS',
  2: 'ADGD',
  3: 'HHA',
  4: 'HAKD',
  5: 'HADSA'},
 'date_activated': {0: Timestamp('2021-01-05 00:00:00'),
  1: Timestamp('2020-12-12 00:00:00'),
  2: Timestamp('2021-02-03 00:00:00'),
  3: Timestamp('2021-03-05 00:00:00'),
  4: Timestamp('2021-03-06 00:00:00'),
  5: Timestamp('2021-05-01 00:00:00')}}
df=pd.DataFrame(d)
df

,account_id,contract_id,date_activated
0,1,AAA,2021-01-05
1,1,ADS,2020-12-12
2,1,ADGD,2021-02-03
3,2,HHA,2021-03-05
4,2,HAKD,2021-03-06
5,3,HADSA,2021-05-01


In [74]:
d={'account_id': {0: 1, 1: 1, 2: 1, 3: 2, 4: 2, 5: 3},
 'contract_id': {0: 'ADS',
  1: 'AAA',
  2: 'ADGD',
  3: 'HHA',
  4: 'HAKD',
  5: 'HADSA'},
 'date_activated': {0: Timestamp('2020-12-12 00:00:00'),
  1: Timestamp('2021-01-05 00:00:00'),
  2: Timestamp('2021-02-03 00:00:00'),
  3: Timestamp('2021-03-05 00:00:00'),
  4: Timestamp('2021-03-06 00:00:00'),
  5: Timestamp('2021-05-01 00:00:00')},
 'Renewal Order': {0: 'Original',
  1: '1st',
  2: '2nd',
  3: 'Original',
  4: '1st',
  5: 'Original'}}
df = pd.DataFrame(d)
df

,account_id,contract_id,date_activated,Renewal Order
0,1,ADS,2020-12-12,Original
1,1,AAA,2021-01-05,1st
2,1,ADGD,2021-02-03,2nd
3,2,HHA,2021-03-05,Original
4,2,HAKD,2021-03-06,1st
5,3,HADSA,2021-05-01,Original


In [81]:
df = df.sort_values(['account_id', 'date_activated']).reset_index(drop=True)
df['Renewal Order'] = df.groupby('account_id').cumcount()
conditions = [
    df['Renewal Order']==0,
    df['Renewal Order']==1,
    df['Renewal Order']==2
]
choices = ['Original', '1st', '2nd']
df['Renewal Order'] = np.select(conditions, choices, default='black')
df

In [103]:
d="""col1,col2,col3,col4
A,2021-03-28 01:40:00,1.381158,0.0
A,2021-03-28 01:50:00,0.480089,0.0
A,2021-03-28 03:00:00,0.000000,0.0
A,2021-03-28 03:00:00,0.111088,0.0
A,2021-03-28 03:10:00,0.000000,0.0
A,2021-03-28 03:10:00,0.000000,0.0
A,2021-03-28 03:10:00,0.151066,0.0
B,2021-03-28 03:10:00,1.231341,1.0"""
df=g(d)
df

,col1,col2,col3,col4
0,A,2021-03-28 01:40:00,1.381158,0.0
1,A,2021-03-28 01:50:00,0.480089,0.0
2,A,2021-03-28 03:00:00,0.000000,0.0
3,A,2021-03-28 03:00:00,0.111088,0.0
4,A,2021-03-28 03:10:00,0.000000,0.0
5,A,2021-03-28 03:10:00,0.000000,0.0
6,A,2021-03-28 03:10:00,0.151066,0.0
7,B,2021-03-28 03:10:00,1.231341,1.0


In [104]:
df.col2 = pd.to_datetime(df.col2)
df

,col1,col2,col3,col4
0,A,2021-03-28 01:40:00,1.381158,0.0
1,A,2021-03-28 01:50:00,0.480089,0.0
2,A,2021-03-28 03:00:00,0.000000,0.0
3,A,2021-03-28 03:00:00,0.111088,0.0
4,A,2021-03-28 03:10:00,0.000000,0.0
5,A,2021-03-28 03:10:00,0.000000,0.0
6,A,2021-03-28 03:10:00,0.151066,0.0
7,B,2021-03-28 03:10:00,1.231341,1.0


In [111]:
df.groupby(['col1', 'col2'])['col3']